In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import plot_model

import matplotlib.pyplot as plt

import sys

import numpy as np

class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()
        plot_model(model, show_shapes= True, to_file='generator.png')

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()
        plot_model(model,show_shapes= True, to_file='discriminator.png')

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=4000, batch_size=32, save_interval=50)


C:\Users\adward\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
__________

C:\Users\adward\Anaconda3\lib\site-packages\keras\engine\training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.059193, acc.: 35.94%] [G loss: 0.698001]


FileNotFoundError: [Errno 2] No such file or directory: 'images/mnist_0.png'

## 1、导入keras及可视化相关包

In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import plot_model

import matplotlib.pyplot as plt
import sys
import numpy as np

C:\Users\adward\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 2、设计主函数

In [2]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()
        plot_model(model, show_shapes= True, to_file='generator.png')

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()
        plot_model(model,show_shapes= True, to_file='discriminator.png')

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()

## 3、运行

In [3]:
if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=4000, batch_size=32, save_interval=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
__________

C:\Users\adward\Anaconda3\lib\site-packages\keras\engine\training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.223735, acc.: 39.06%] [G loss: 0.575408]
1 [D loss: 0.788143, acc.: 54.69%] [G loss: 0.903763]
2 [D loss: 0.639030, acc.: 73.44%] [G loss: 1.429563]
3 [D loss: 0.729423, acc.: 60.94%] [G loss: 1.278944]
4 [D loss: 0.572790, acc.: 75.00%] [G loss: 1.470756]
5 [D loss: 0.557676, acc.: 67.19%] [G loss: 1.184316]
6 [D loss: 0.416351, acc.: 84.38%] [G loss: 0.731446]
7 [D loss: 0.377336, acc.: 81.25%] [G loss: 0.725438]
8 [D loss: 0.391577, acc.: 85.94%] [G loss: 0.710275]
9 [D loss: 0.554773, acc.: 78.12%] [G loss: 1.069237]
10 [D loss: 0.752420, acc.: 56.25%] [G loss: 1.426523]
11 [D loss: 0.769296, acc.: 54.69%] [G loss: 1.433455]
12 [D loss: 0.844961, acc.: 53.12%] [G loss: 1.440901]
13 [D loss: 0.533461, acc.: 79.69%] [G loss: 1.255348]
14 [D loss: 0.649969, acc.: 67.19%] [G loss: 1.034229]
15 [D loss: 0.849641, acc.: 59.38%] [G loss: 1.177589]
16 [D loss: 0.874995, acc.: 56.25%] [G loss: 1.556774]
17 [D loss: 0.869787, acc.: 48.44%] [G loss: 1.436302]
18 [D loss: 0.740346

149 [D loss: 0.632749, acc.: 60.94%] [G loss: 1.131588]
150 [D loss: 0.906394, acc.: 46.88%] [G loss: 1.132839]
151 [D loss: 0.931546, acc.: 39.06%] [G loss: 1.030578]
152 [D loss: 0.989454, acc.: 37.50%] [G loss: 1.107707]
153 [D loss: 0.771382, acc.: 57.81%] [G loss: 1.402595]
154 [D loss: 0.863008, acc.: 46.88%] [G loss: 0.988737]
155 [D loss: 0.847528, acc.: 53.12%] [G loss: 1.053684]
156 [D loss: 0.788094, acc.: 45.31%] [G loss: 1.244722]
157 [D loss: 0.889254, acc.: 43.75%] [G loss: 1.024391]
158 [D loss: 1.020349, acc.: 34.38%] [G loss: 0.812468]
159 [D loss: 0.914325, acc.: 34.38%] [G loss: 0.879642]
160 [D loss: 0.862734, acc.: 35.94%] [G loss: 0.861392]
161 [D loss: 0.856399, acc.: 40.62%] [G loss: 1.019312]
162 [D loss: 0.748838, acc.: 53.12%] [G loss: 1.141544]
163 [D loss: 0.906845, acc.: 43.75%] [G loss: 0.930292]
164 [D loss: 0.756086, acc.: 51.56%] [G loss: 0.908151]
165 [D loss: 0.897371, acc.: 50.00%] [G loss: 1.115453]
166 [D loss: 0.887224, acc.: 34.38%] [G loss: 1.

296 [D loss: 0.841287, acc.: 46.88%] [G loss: 1.009384]
297 [D loss: 0.731119, acc.: 50.00%] [G loss: 1.186830]
298 [D loss: 0.854313, acc.: 45.31%] [G loss: 1.098560]
299 [D loss: 0.909981, acc.: 39.06%] [G loss: 0.858140]
300 [D loss: 0.867900, acc.: 40.62%] [G loss: 0.889081]
301 [D loss: 0.790754, acc.: 43.75%] [G loss: 1.011689]
302 [D loss: 0.729539, acc.: 50.00%] [G loss: 1.138627]
303 [D loss: 0.831595, acc.: 46.88%] [G loss: 0.820472]
304 [D loss: 0.794452, acc.: 48.44%] [G loss: 0.964688]
305 [D loss: 0.797440, acc.: 45.31%] [G loss: 1.014983]
306 [D loss: 0.791614, acc.: 51.56%] [G loss: 0.899551]
307 [D loss: 0.972992, acc.: 35.94%] [G loss: 0.861008]
308 [D loss: 0.816430, acc.: 45.31%] [G loss: 0.881224]
309 [D loss: 0.720009, acc.: 57.81%] [G loss: 0.882268]
310 [D loss: 0.792606, acc.: 48.44%] [G loss: 0.995002]
311 [D loss: 0.699912, acc.: 56.25%] [G loss: 1.089018]
312 [D loss: 0.794297, acc.: 51.56%] [G loss: 0.879455]
313 [D loss: 0.782651, acc.: 53.12%] [G loss: 1.

...

3334 [D loss: 0.591288, acc.: 65.62%] [G loss: 1.157281]
3335 [D loss: 0.576260, acc.: 70.31%] [G loss: 1.025772]
3336 [D loss: 0.677399, acc.: 64.06%] [G loss: 0.856770]
3337 [D loss: 0.646856, acc.: 64.06%] [G loss: 1.068846]
3338 [D loss: 0.679162, acc.: 59.38%] [G loss: 0.830879]
3339 [D loss: 0.551811, acc.: 71.88%] [G loss: 1.101057]
3340 [D loss: 0.591410, acc.: 70.31%] [G loss: 1.118606]
3341 [D loss: 0.661164, acc.: 65.62%] [G loss: 0.902109]
3342 [D loss: 0.685103, acc.: 64.06%] [G loss: 1.026485]
3343 [D loss: 0.539032, acc.: 75.00%] [G loss: 1.100123]
3344 [D loss: 0.585778, acc.: 73.44%] [G loss: 1.104641]
3345 [D loss: 0.690543, acc.: 54.69%] [G loss: 0.997372]
3346 [D loss: 0.658719, acc.: 54.69%] [G loss: 0.967400]
3347 [D loss: 0.657798, acc.: 62.50%] [G loss: 1.102863]
3348 [D loss: 0.742523, acc.: 51.56%] [G loss: 1.003314]
3349 [D loss: 0.715805, acc.: 53.12%] [G loss: 1.004911]
3350 [D loss: 0.741050, acc.: 54.69%] [G loss: 0.922351]
3351 [D loss: 0.690804, acc.: 6

3478 [D loss: 0.672251, acc.: 54.69%] [G loss: 1.008398]
3479 [D loss: 0.635486, acc.: 68.75%] [G loss: 0.961685]
3480 [D loss: 0.741219, acc.: 54.69%] [G loss: 1.061063]
3481 [D loss: 0.687114, acc.: 60.94%] [G loss: 0.986861]
3482 [D loss: 0.650583, acc.: 65.62%] [G loss: 0.985165]
3483 [D loss: 0.662906, acc.: 67.19%] [G loss: 0.975742]
3484 [D loss: 0.616198, acc.: 60.94%] [G loss: 0.952664]
3485 [D loss: 0.707090, acc.: 56.25%] [G loss: 0.908002]
3486 [D loss: 0.619547, acc.: 64.06%] [G loss: 0.996524]
3487 [D loss: 0.642712, acc.: 62.50%] [G loss: 1.033224]
3488 [D loss: 0.663211, acc.: 60.94%] [G loss: 0.848480]
3489 [D loss: 0.781663, acc.: 50.00%] [G loss: 1.094058]
3490 [D loss: 0.718004, acc.: 54.69%] [G loss: 0.824369]
3491 [D loss: 0.675887, acc.: 54.69%] [G loss: 1.087641]
3492 [D loss: 0.774032, acc.: 48.44%] [G loss: 0.910513]
3493 [D loss: 0.645889, acc.: 64.06%] [G loss: 1.036940]
3494 [D loss: 0.709333, acc.: 57.81%] [G loss: 0.978267]
3495 [D loss: 0.682639, acc.: 5

3622 [D loss: 0.690902, acc.: 59.38%] [G loss: 1.029048]
3623 [D loss: 0.787005, acc.: 46.88%] [G loss: 0.879961]
3624 [D loss: 0.607951, acc.: 67.19%] [G loss: 1.162199]
3625 [D loss: 0.624815, acc.: 68.75%] [G loss: 0.968595]
3626 [D loss: 0.728467, acc.: 57.81%] [G loss: 1.102347]
3627 [D loss: 0.637754, acc.: 64.06%] [G loss: 0.960778]
3628 [D loss: 0.705743, acc.: 57.81%] [G loss: 0.838664]
3629 [D loss: 0.630997, acc.: 60.94%] [G loss: 0.975498]
3630 [D loss: 0.716729, acc.: 57.81%] [G loss: 0.920583]
3631 [D loss: 0.653502, acc.: 57.81%] [G loss: 1.134408]
3632 [D loss: 0.730103, acc.: 53.12%] [G loss: 0.927905]
3633 [D loss: 0.692109, acc.: 60.94%] [G loss: 1.052189]
3634 [D loss: 0.665622, acc.: 57.81%] [G loss: 1.041656]
3635 [D loss: 0.606719, acc.: 71.88%] [G loss: 0.907876]
3636 [D loss: 0.685412, acc.: 60.94%] [G loss: 0.983115]
3637 [D loss: 0.721543, acc.: 54.69%] [G loss: 0.948070]
3638 [D loss: 0.737393, acc.: 48.44%] [G loss: 0.928361]
3639 [D loss: 0.612392, acc.: 6

3766 [D loss: 0.710019, acc.: 54.69%] [G loss: 0.963320]
3767 [D loss: 0.611820, acc.: 67.19%] [G loss: 1.091932]
3768 [D loss: 0.699315, acc.: 51.56%] [G loss: 0.993462]
3769 [D loss: 0.626349, acc.: 62.50%] [G loss: 0.928941]
3770 [D loss: 0.715368, acc.: 50.00%] [G loss: 0.993965]
3771 [D loss: 0.738669, acc.: 43.75%] [G loss: 0.826055]
3772 [D loss: 0.645862, acc.: 59.38%] [G loss: 1.033696]
3773 [D loss: 0.650761, acc.: 56.25%] [G loss: 0.896529]
3774 [D loss: 0.678364, acc.: 60.94%] [G loss: 0.911834]
3775 [D loss: 0.670183, acc.: 57.81%] [G loss: 0.915951]
3776 [D loss: 0.649617, acc.: 65.62%] [G loss: 0.958327]
3777 [D loss: 0.630856, acc.: 68.75%] [G loss: 1.104285]
3778 [D loss: 0.660109, acc.: 59.38%] [G loss: 0.986140]
3779 [D loss: 0.630767, acc.: 60.94%] [G loss: 1.094335]
3780 [D loss: 0.661350, acc.: 59.38%] [G loss: 0.947758]
3781 [D loss: 0.585545, acc.: 71.88%] [G loss: 1.011256]
3782 [D loss: 0.678039, acc.: 57.81%] [G loss: 0.855331]
3783 [D loss: 0.676956, acc.: 5

3910 [D loss: 0.779609, acc.: 51.56%] [G loss: 0.923744]
3911 [D loss: 0.629366, acc.: 68.75%] [G loss: 0.846922]
3912 [D loss: 0.655486, acc.: 60.94%] [G loss: 0.825171]
3913 [D loss: 0.717173, acc.: 51.56%] [G loss: 0.852024]
3914 [D loss: 0.615361, acc.: 62.50%] [G loss: 0.967192]
3915 [D loss: 0.710413, acc.: 51.56%] [G loss: 0.929077]
3916 [D loss: 0.653018, acc.: 59.38%] [G loss: 0.999570]
3917 [D loss: 0.639391, acc.: 67.19%] [G loss: 1.072105]
3918 [D loss: 0.698931, acc.: 53.12%] [G loss: 1.016743]
3919 [D loss: 0.730808, acc.: 51.56%] [G loss: 1.028825]
3920 [D loss: 0.756421, acc.: 53.12%] [G loss: 1.121961]
3921 [D loss: 0.617485, acc.: 68.75%] [G loss: 1.008327]
3922 [D loss: 0.704223, acc.: 56.25%] [G loss: 0.934271]
3923 [D loss: 0.674724, acc.: 65.62%] [G loss: 0.950272]
3924 [D loss: 0.665172, acc.: 59.38%] [G loss: 0.928980]
3925 [D loss: 0.664618, acc.: 53.12%] [G loss: 0.868897]
3926 [D loss: 0.587963, acc.: 70.31%] [G loss: 1.015017]
3927 [D loss: 0.704726, acc.: 5